In [1]:
import WELdatalake as wdl
import pandas as pd
import os

Environment set to "prod".
Authentication enabled.


In [2]:
COLUMN_MAPPING = {
    "work_order_number": "id",
    "work_order_text": "description",
    "work_order_type_code": "wo_order_type",
    "work_order_actual_cost": "total_actual_costs",
    "work_order_actual_start_date": "actual_start_date",
    "work_order_actual_finish_date": "actual_finish_date",
    "functional_location_text" : "functional_loc_desc",
    "functional_location_code": "functional_loc"
}

In [3]:
def extract():
    return wdl.custom.plant_maintenance_get(
        fields = list(COLUMN_MAPPING.keys()),
        filter = "FUNCTIONAL_LOCATION_OBJECT_TYPE_CODE = 'MEFU' AND audit_record_active_flag = 'Y'"
        )
data = extract()

2022-09-09 10:02:45 [INFO] Sending POST request to https://api-ukniteedata.data.woodside/api/v2/snowflake/statements
2022-09-09 10:02:49 [INFO] Sending GET request to https://api-ukniteedata.data.woodside/api/v2/snowflake/statements/01a6d7fa-3201-1508-0000-6d35094ab936?partition=1
/home/wde.woodside.com.au/w47077/.local/share/virtualenvs/ds-reliability-1-zAS3XUEA/lib/python3.8/site-packages/WELdatalake/_ukniteeDataApi.py:341: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  allData = allData.append(data, ignore_index=True)


In [4]:
def transform(df: pd.DataFrame):
    data = df.copy()
    data.columns = data.columns.str.lower()
    data.rename(
        columns = COLUMN_MAPPING,
        inplace = True
    )

    return data

data = transform(data)
data.columns

Index(['id', 'description', 'wo_order_type', 'total_actual_costs',
       'actual_start_date', 'actual_finish_date', 'functional_loc_desc',
       'functional_loc'],
      dtype='object')

In [5]:
def load(df: pd.DataFrame, name:str,):
    data.to_csv(
        os.path.join("..","data",name)
        )
load(data, "MEFU.csv")